In [1]:
!pip install pip --upgrade
!pip install Wikipedia-API
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 24.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13476 sha256=2ba9e091aca9cb4a62fb2c280ab9c6606a5bd78f8ae258b37833e8f970b999f0
  Stored in directory: /root/.cache/pip/wheels/55/bd/a2/b9e73c95f4ea806361f98dc34bce6d1a515c2fc06588d4e1bf
Successfully built Wikipedia-API
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-py

In [13]:
import re
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import wikipediaapi
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Top 100 grossing movie list dataset




## Making a movie list by scraping the wikipedia top grossing list webpages

In [3]:
# Making a movie list by scraping the wikipedia top grossing list webpages

def getMovies(url):
  request = requests.get(url)
  #gets the content from the web site
  src = request.content
  #used lxml to parse the data
  soup = BeautifulSoup(src,'lxml')
  #Gets the table with the class 'wikitable'
  movies_table = soup.find('table',{'class':'wikitable'})
  #Gets all items with the tag 'a'
  raw_names = movies_table.find_all("a")
  names = []

  for item in raw_names:
      name = item.get('title')
      if name != None and not re.match(r"^\d{4}", name):
          names.append(name)
  return names


url1 ="https://en.wikipedia.org/wiki/List_of_highest-grossing_films_in_the_United_States_and_Canada"
url2 = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

names1 = getMovies(url1)
# print(len(names1))
names2 = getMovies(url2)
# print(len(names1))
names = list(set(names1+names2)) # keeping only the unique movies
print("Got the names of 106 movies:",len(names))

Got the names of 106 movies: 106


In [4]:
names

['Jumanji: The Next Level',
 'Thor: Ragnarok',
 'The Lion King',
 "Harry Potter and the Philosopher's Stone (film)",
 'Avengers: Endgame',
 'Jumanji: Welcome to the Jungle',
 'The Lord of the Rings: The Two Towers',
 'Frozen II',
 'Shrek the Third',
 'The Hobbit: An Unexpected Journey',
 'Star Wars: The Force Awakens',
 'The Lord of the Rings: The Fellowship of the Ring',
 'It (2017 film)',
 'Jurassic World Dominion',
 'The Passion of the Christ',
 'Star Wars: Episode III – Revenge of the Sith',
 'Top Gun: Maverick',
 'The Hunger Games: Mockingjay – Part 1',
 'Minions: The Rise of Gru',
 'Minions (film)',
 'Despicable Me 3',
 'Guardians of the Galaxy Vol. 2',
 'Harry Potter and the Deathly Hallows – Part 1',
 'Avengers: Age of Ultron',
 'The Dark Knight Rises',
 'Rogue One: A Star Wars Story',
 'Transformers: Age of Extinction',
 'Spider-Man 2',
 'Toy Story 4',
 'Toy Story 3',
 'Forrest Gump',
 'Spider-Man 3',
 'Beauty and the Beast (2017 film)',
 "Pirates of the Caribbean: At World's 

## Retriving plots of movies by searching their name on wikipedia-api

In [ ]:
# Movies: names list
# Plot: plot text
wiki = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
# wiki = wikipediaapi.Wikipedia(language='en')

def getPlots(names):
  plots = []
  def storePlot(sections, level=0):
          for s in sections:
            if s.title == 'Plot':
              plot=[]
              for p in s.text.split("\n"):
                plot.append(p)
              plots.append(plot)
  for name in names:
    page_py = wiki.page(name)
    storePlot(page_py.sections)
  return plots

plots = getPlots(names)

In [9]:
plots

[]

## Storing top grossing movie plot list which has paragraph as scenes

In [ ]:
json_dict = {"Movies": names, "Plots": plots}
print(len(json_dict["Movies"]))
with open("/data/paraWikiTopGrossing.txt", "w", encoding="UTF-8") as fp:
    json.dump(json_dict, fp)

## Storing top grossing movie plot list which has sentence as scenes

In [10]:
# Changing plot which contains paragraph as scenes to sentence as scenes
from nltk import sent_tokenize
sentPlots=[]
for x in plots:
  res=[]
  for p in x:
      res+=sent_tokenize(p)
  sentPlots.append(res)

In [ ]:
json_dict = {"Movies": names, "Plots": sentPlots}
# print(len(json_dict["Movies"]))
with open("/data/sentenceWikiTopGrossing.txt", "w", encoding="UTF-8") as fp:
    json.dump(json_dict, fp)

106


# Tip Of My Tongue (TOMT) movie list dataset

In [15]:
# wikipedia links
links = [["https://en.wikipedia.org/wiki/Twisted_(1986_film)"],
["https://en.wikipedia.org/wiki/Chatroom_(film)"],
["https://en.wikipedia.org/wiki/The_10th_Kingdom"],
["https://en.wikipedia.org/wiki/The_Dragon_Ring"],
["https://en.wikipedia.org/wiki/Nowhere_(film)"],
["https://en.wikipedia.org/wiki/Dead_End_(2003_film)"],
["https://en.wikipedia.org/wiki/The_Warlord:_Battle_for_the_Galaxy"],
["https://en.wikipedia.org/wiki/The_Final_(film)"],
["https://en.wikipedia.org/wiki/Tales_of_the_Night_(film)"],
["https://en.wikipedia.org/wiki/The_Watcher_in_the_Woods_(1980_film)"]]

In [16]:
# Getting movie names from wikipedia links
movnames = [x[0].rsplit("/",1)[1] for x in links]
print(movnames)
print(len(movnames))
print(len(links))

['Twisted_(1986_film)', 'Chatroom_(film)', 'The_10th_Kingdom', 'The_Dragon_Ring', 'Nowhere_(film)', 'Dead_End_(2003_film)', 'The_Warlord:_Battle_for_the_Galaxy', 'The_Final_(film)', 'Tales_of_the_Night_(film)', 'The_Watcher_in_the_Woods_(1980_film)']
10
10


In [ ]:
movnames

['Twisted_(1986_film)',
 'Chatroom_(film)',
 'The_10th_Kingdom',
 'The_Dragon_Ring',
 'Nowhere_(film)',
 'Dead_End_(2003_film)',
 'The_Warlord:_Battle_for_the_Galaxy',
 'The_Final_(film)',
 'Tales_of_the_Night_(film)',
 'The_Watcher_in_the_Woods_(1980_film)']

## Retriving plots of movies by their name on wikipedia-api

In [ ]:
# Movies: names list
# Plot: plot text


def getPlots(names):
  plots = []
  def storePlot(sections, level=0):
          for s in sections:
            if s.title == 'Plot':
              plot=[]
              # print("%s" % (s.title))
              for p in s.text.split("\n"):
                plot.append(p)
              plots.append(plot)
  for name in names:
    page_py = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI).page(name)
    storePlot(page_py.sections)
  return plots

plots = getPlots(movnames)

## Storing TOMT movie plot list which has paragraph as scenes

In [ ]:
json_dict = {"Movies": movnames, "Plots": plots}
print(len(json_dict["Movies"]))
with open("/data/paraWikiScrapped.txt", "w", encoding="UTF-8") as fp:
    json.dump(json_dict, fp)

10


In [ ]:
with open("/data/paraWikiScrapped.txt", "r", encoding="UTF-8") as f:
    json_data = json.load(f)

Movies, Plots = json_data["Movies"],json_data["Plots"]

# print(Movies[0],Plots[0])
for x in plots:
  for p in x:
    print(p)
  print("--------------------------------------------------------------")

One evening, the Collins, Phillip and Evelyn, a married couple with two children, discover their maid, Mrs. Murdock, dead at the end of their steps; her neck is broken. Evidently, she had an accident; now they need a new babysitter for an upcoming party. The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job. She does not know Susan's teenage brother Mark (Christian Slater): technically skilled and good in school, but restive and cunning. Mark also listens regularly to German marching music from the Third Reich. As soon as the parents have left, he psychologically terrorizes Helen and his sister with electronic tricks. Williams (Karl Taylor), a school jock whom Mark burned earlier in science class, comes to the Collins home out for revenge; Mark kills him with a fencing sword.
Ultimately, Mark himself is killed when Helen, defending herself, knocks him onto his spiked German helmet. Mark's parents come home to find the house in s

## Storing TOMT movie plot list which has sentence as scenes

In [ ]:
# Changing plot which contains paragraph as scenes to sentence as scenes
from nltk import sent_tokenize
sentPlots=[]

for x in plots:
  res=[]
  for p in x:
      res+=sent_tokenize(p)
  sentPlots.append(res)

In [ ]:
for p in sentPlots:
  print(len(p))

12
31
25
11
23
62
51
46
3
23


In [ ]:
# printing sentence plots
i=0
index=0
for x in sentPlots:
  print(movnames[i])
  for p in x:
    print(p)
  if movnames[i]=="Tales_of_the_Night_(film)":
    index=i
  i+=1
  print("--------------------------------------------------------------")
print(index)

Twisted_(1986_film)
One evening, the Collins, Phillip and Evelyn, a married couple with two children, discover their maid, Mrs. Murdock, dead at the end of their steps; her neck is broken.
Evidently, she had an accident; now they need a new babysitter for an upcoming party.
The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.
She does not know Susan's teenage brother Mark (Christian Slater): technically skilled and good in school, but restive and cunning.
Mark also listens regularly to German marching music from the Third Reich.
As soon as the parents have left, he psychologically terrorizes Helen and his sister with electronic tricks.
Williams (Karl Taylor), a school jock whom Mark burned earlier in science class, comes to the Collins home out for revenge; Mark kills him with a fencing sword.
Ultimately, Mark himself is killed when Helen, defending herself, knocks him onto his spiked German helmet.
Mark's parents come home to

In [ ]:
import json
json_dict = {"Movies": movnames, "Plots": sentPlots}
print(len(json_dict["Movies"]))
with open("/data/sentenceWikiScrapped.txt", "w", encoding="UTF-8") as fp:
    json.dump(json_dict, fp)

10


In [ ]:
with open("/data/sentenceWikiScrapped.txt", "r", encoding="UTF-8") as f:
    json_data = json.load(f)

Movies, Plots = json_data["Movies"],json_data["Plots"]

# print(Movies[0],Plots[0])
for x in Plots:
  for p in x:
    print(p)
  print("--------------------------------------------------------------")
  # ['Twisted_(1986_film)', 'Chatroom_(film)', 'The_10th_Kingdom', 'The_Dragon_Ring', 'Nowhere_(film)', 'Dead_End_(2003_film)', 'The_Warlord:_Battle_for_the_Galaxy', 'The_Final_(film)', 'Tales_of_the_Night_(film)', 'The_Watcher_in_the_Woods_(1980_film)']

One evening, the Collins, Phillip and Evelyn, a married couple with two children, discover their maid, Mrs. Murdock, dead at the end of their steps; her neck is broken.
Evidently, she had an accident; now they need a new babysitter for an upcoming party.
The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.
She does not know Susan's teenage brother Mark (Christian Slater): technically skilled and good in school, but restive and cunning.
Mark also listens regularly to German marching music from the Third Reich.
As soon as the parents have left, he psychologically terrorizes Helen and his sister with electronic tricks.
Williams (Karl Taylor), a school jock whom Mark burned earlier in science class, comes to the Collins home out for revenge; Mark kills him with a fencing sword.
Ultimately, Mark himself is killed when Helen, defending herself, knocks him onto his spiked German helmet.
Mark's parents come home to find the house in s